# UKWA Stats Module

Copyright 2023 Michael George (AKA Logiqx).

This file is part of [wsw-results](https://github.com/Logiqx/wsw-results) and is distributed under the terms of the GNU General Public License.

wsw-results is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

wsw-results is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with sse-results. If not, see <https://www.gnu.org/licenses/>.

### Summary

Adhoc script to determine the correctness of UKWA flags in the current year

## Initialisation

Basic approach to determine the project directory

In [1]:
import os
import sys
import glob

import csv
import math

import numpy as np

from datetime import datetime

In [2]:
PROJDIR = os.path.realpath(os.path.join(sys.path[0], '..'))

## Process Entrants

Process all names

In [3]:
def processSession(year, csvFile):
    '''processSession processes an individual session for an event'''

    session = os.path.basename(os.path.dirname(csvFile))
    
    RUN_2_KTS = 'Run 2 (kts)'
    AVG_KTS = 'Avg (kts)'

    results = []
    runs = []
    averages = []
    with open(csvFile, 'r', encoding='utf-8') as f:
        csvReader = csv.DictReader(f)
        for result in csvReader:
            results.append(result)
            try:
                runs.append(float(result[RUN_2_KTS]))
            except ValueError:
                runs.append(0)
            averages.append(float(result[AVG_KTS]))
            
    runs = sorted(runs, reverse=True)
            
    percent25 = math.ceil(len(runs) / 4)
    percent50 = math.ceil(len(runs) / 2)
    
    if runs[percent25 - 1] >= 28 and runs[percent50 - 1] >= 26:
        #print(session, 'valid-both', len(results), runs[percent25 - 1], runs[percent50 - 1])
        pass

    if runs[percent50 - 1]:
        if runs[percent25 - 1] >= 28:
            #print(session, 'valid-25', len(results), runs[percent25 - 1], runs[percent50 - 1])
            pass
        elif runs[percent50 - 1] >= 26:
            print(session, 'valid-50', len(results), runs[percent25 - 1], runs[percent50 - 1])
            pass
        else:
            #print(session, 'invalid', len(results), runs[percent25 - 1], runs[percent50 - 1])
            pass

    if runs[percent25 - 1]:
        if runs[percent50 - 1] >= 26:
            #print(session, 'valid-50', len(results), runs[percent25 - 1], runs[percent50 - 1])
            pass
        elif runs[percent25 - 1] >= 28:
            print(session, 'valid-25', len(results), runs[percent25 - 1], runs[percent50 - 1])
        else:
            #print(session, 'invalid', len(results), runs[percent25 - 1], runs[percent50 - 1])
            pass

In [4]:
def processEvent(year):
    '''processYear processes all of the sessions for an event'''

    csvFiles = sorted(glob.glob(os.path.join(PROJDIR, 'events', str(year), 'sailwave', '*', '*Men*.csv')))
    
    for csvFile in csvFiles:
        if "Wing" not in csvFile:
            processSession(year, csvFile)

In [5]:
def main():
    for year in range(2000, 2050):
        processEvent(year)
      
    print('All done')

if __name__ == '__main__':
    main()

20111019 valid-50 18 26.42 26.09
20121012 valid-25 24 28.12 25.34
20151005 valid-50 24 27.51 26.04
20181013 valid-25 25 28.95 25.62
20181018 valid-25 19 28.32 25.76
20221019 valid-25 11 30.28 22.86
20231011 valid-50 25 27.51 26.25
All done


## All Done!